In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import os

### 获取数据

In [2]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()),"data.db")
engine_path = "sqlite:///"+db_path

# 获取数据函数，根据输入的SQL语句返回 DataFrame 类型数据
def link_sqlite(sql):
    engine = create_engine(engine_path)
    df = pd.read_sql(sql,con=engine)
    return df

### 线性数据预测

In [4]:
sql = "select * from predictSalesSummary where shopid=1"
df = link_sqlite(sql)

In [5]:
df.tail()

,month,amount,shopid
7,2021年6月,23181.5,1
8,2021年7月,27703.4,1
9,2021年8月,33041.2,1
10,2021年9月,38391.9,1
11,2021年10月,44715.2,1


In [6]:
from pyecharts import options as opts
from pyecharts.charts import Bar

x_names = df["month"].tolist()
tao_bao = df["amount"].tolist()

c = (
    Bar()
    .add_xaxis(x_names)
    .add_yaxis("销售额", tao_bao)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=20)),
        title_opts=opts.TitleOpts(title="商品每个月销售额"),
    )
)
c.render_notebook()

In [7]:
# 从2021年2月份开始，数据呈现上升的线性趋势
df2 = df.iloc[3:,:]

# 数据复制一份，避免操作失误导致数据受损
df3 = df2.copy()

df3["x"] = list(range(2,11))

In [10]:
df3

,month,amount,shopid,x
3,2021年2月,6644.7,1,2
4,2021年3月,10639.1,1,3
5,2021年4月,14498.7,1,4
6,2021年5月,18660.9,1,5
7,2021年6月,23181.5,1,6
8,2021年7月,27703.4,1,7
9,2021年8月,33041.2,1,8
10,2021年9月,38391.9,1,9
11,2021年10月,44715.2,1,10


In [11]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,r2_score

In [12]:
# 自变量数据
x = df3["x"].values.tolist()
# 因变量数据
y = df3["amount"].values.tolist()
# 由于自变量只有一个维度，需要改变一下数据结构
x_reshape = np.array(x).reshape(-1,1)
# 实例化一个线性模型
lr = linear_model.LinearRegression()
# 训练数据
lr.fit(x_reshape,y)
# 预测数据
x_predict = np.array([11]).reshape(-1,1)
lr.predict(x_predict)
# array([47636.39166667])

array([47636.39166667])

### 非线性数据预测

In [13]:
shop_2_sql = "select * from predictSalesSummary where shopid=2"
shop_2_df = link_sqlite(shop_2_sql)

In [14]:
shop_2_df

,month,amount,shopid
0,2021年4月,4174.2,2
1,2021年5月,3388.4,2
2,2021年6月,1979.6,2
3,2021年7月,2341.5,2
4,2021年8月,2219.2,2
5,2021年9月,1972.3,2
6,2021年10月,2091.9,2


In [15]:
from pyecharts import options as opts
from pyecharts.charts import Bar

x_names = shop_2_df["month"].tolist()
tao_bao = shop_2_df["amount"].tolist()

c = (
    Bar()
    .add_xaxis(x_names)
    .add_yaxis("销售额", tao_bao)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=20)),
        title_opts=opts.TitleOpts(title="商品每个月销售额"),
    )
)
c.render_notebook()

商品2的销售额，在最近5个月内变化幅度不大，可以用加权平均值的方法预测下个月销售额

In [16]:
np.sum(shop_2_df.iloc[4:,1]*np.array([0.2,0.2,0.6]))

2093.44

In [17]:
shop_2_df.iloc[4:,1].mean()

2094.4666666666667